## Evaluate

Here we use the compiled responses we got from ChatGPT and OpenAssistant to evaluate their LU suggestions

In [ ]:
import os
import json
import pandas as pd

ASSISTANT = "ChatGPT" # OpenAssistant
LANG = "en" # or pt

with open(os.path.join(ASSISTANT, f'event_{LANG}.json')) as fp:
    event_lus = json.load(fp)
    
with open(os.path.join(ASSISTANT, f'entity_{LANG}.json')) as fp:
    entity_lus = json.load(fp)
    
with open(os.path.join(ASSISTANT, f'attribute_{LANG}.json')) as fp:
    attribute_lus = json.load(fp)
    
# with open(os.path.join(ASSISTANT, f'artifact_{LANG}.json')) as fp:
#     artifact_lus = json.load(fp)

In [ ]:
def create_df(type_lus):
    return pd.DataFrame.from_records([
        { "idFrame": int(idFrame), "lu": lu.lower().split('.')[0], "split": split }
        for split, frames in type_lus.items()
        for idFrame, lus in frames.items()
        for lu in lus
    ])


def is_in_fn_plus(row):
    frame_id = row["idFrame"]
    return frame_id in fn_plus and row["lu"] in fn_plus[frame_id]


def is_in_fnbr(row):
    frame_id = row["idFrame"]
    return frame_id in fnbr and row["lu"] in fnbr[frame_id]

In [ ]:
df = create_df(event_lus)

df['split'] = df['split'].str.replace('split-', '')

if LANG == "en":
    df['FN+?'] = df.apply(is_in_fn_plus, axis='columns')
else:
    df['FN-Br?'] = df.apply(is_in_fnbr, axis='columns')

df = df.merge(frames.reset_index(drop=True), on='idFrame')

df[['name', 'lu', 'split', 'FN+?']]

In [ ]:
df['Correct?'] = df.apply(lambda x: x['lu'] in chatgpt_lus, axis='columns')
df[['Correct?']].to_csv('correct.csv', index=False)

In [ ]:
def is_correct(frame, lu):
    row = df_correct[(df_correct['Frame'] == frame) & (df_correct['Lexical Unit'] == lu)]
    
    if len(row) > 0:
        return row.iloc[0]['Correct?']
    
    return False

df_correct = pd.read_csv(
    os.path.join('ChatGPT - Correct', 'ChatGPT_Open Assistant Lexical Units - Events (pt).csv'))

df_correct = df_correct[['Frame', 'Lexical Unit', 'Correct?']]

df['Correct?'] = df.apply(lambda x: is_correct(x['name'], x['lu']), axis='columns')

In [ ]:
df['Correct?'].to_csv('correct.csv')